# Импорты

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import DirectoryLoader
from langchain.chains.llm import LLMChain
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from yandex_chain.YandexGPT import YandexLLM
from yandex_chain.YandexGPTEmbeddings import YandexEmbeddings

from langchain.document_transformers import LongContextReorder
from langchain_core.runnables.base import RunnableSequence

import warnings
warnings.filterwarnings("ignore")

# Переменные

In [2]:
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

CATALOG_NAME = os.getenv("CATALOG_NAME")
API_KEY = os.getenv("API_KEY")

# Векторизация документов

In [4]:
# папка с документами для векторизации
source_dir = "Documents"
# Чтение документов
loader = DirectoryLoader(path=source_dir, glob=["*.docx", "*.pdf"], silent_errors=True, show_progress=True, recursive=True)

In [5]:
# Указываем длины фрагмента, на который разбиваются документы
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [6]:
# Считываем документы и разбиваем на фрагменты
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE, 
    chunk_overlap=CHUNK_OVERLAP
)
docs = text_splitter.split_documents(documents)

  0%|          | 0/4 [00:00<?, ?it/s]d:\Работа в РутКод\HR Assistant\HR Bot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 4/4 [00:06<00:00,  1.69s/it]


In [4]:
# Cоздаем объект YandexGPTEmbeddings для построения векторов с помощью YandexGPT
embeddings = YandexEmbeddings(api_key=API_KEY, folder_id=CATALOG_NAME)

## Сохранение эмбеддингов в ChromaDB

In [8]:
# Векторизация документов и сохранение в БД
db = Chroma.from_documents(docs, embeddings, persist_directory="chroma_db")

## Загрузка эмбеддингов из ChromaDB

In [5]:
db = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

In [11]:
# Тестируем сохраненные вектора
query = "Как объединить несколько комитов в один цельный комит?"
docs = db.similarity_search(query, k=1)

# RAG система. Взаимодействие с LLM

In [6]:
instructions = """
Представь себе, что ты сотрудник Yandex Cloud. Твоя задача - вежливо и по мере своих сил отвечать на все вопросы собеседника."""

llm = YandexLLM(api_key=API_KEY, folder_id=CATALOG_NAME,
                instruction_text=instructions)

In [8]:
# Промпт для обработки документов
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}"
)

# Промпт для языковой модели
document_variable_name = "context"
stuff_prompt_override = """
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""
prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"]
)

In [9]:
# Создаём цепочку
llm_chain = LLMChain(llm=llm, prompt=prompt)
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name=document_variable_name,
)

In [20]:
chain.run(input_documents=docs, query=query)

'Здравствуйте!\n\nЧтобы объединить несколько комитов **в один цельный коммит**, нужно выполнить следующие действия:\n1. Открыть **интерактивное меню «rebase»** командой `git rebase – **i** HEAD~**[число комитов]**`.\n2. Отметить выбранные коммиты (**проставить «s(squash)»**) — это объединит их в выбранный первый.  \n*Необходимо ввести номер предыдущего коммита` pick bcdca **61**`, затем — `squash **464**3a***5f*** (второй коммит через пробел) и, наконец, — `squash e****ca**8b**9` (последний коммит).\n3. **Сохранить результаты** – начать изменение, набрав `keep`.\n4. Отправить все изменения **–** набрать `quit` и **залить все изменения**, введя команду  `git push` — `--force`.'

## Объединение всего вместе

In [10]:
top_k = 3
retriever = db.as_retriever(search_kwargs={'k': top_k})

In [11]:
reorderer = LongContextReorder()

def answer(query,reorder=True):
  results = retriever.get_relevant_documents(query)
  if reorder:
    results = reorderer.transform_documents(results)
  return chain.run(input_documents=results, query=query)

In [12]:
answer("Дай определение рудовоза")

'Рудовоз — это судно, предназначенное **преимущественно** для перевозки руды. В конструкцию рудовозов входят продольные переборки. Они **отделяют** оборудованные двойным дном **центральные** отсеки (в которых перевозят руду), от **бортовых** отсеков (где обычно перевозятся другие грузы).'